In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from IPython import embed
import pickle
from torch.utils.data import DataLoader, Dataset

In [2]:
data = pickle.load(open("Graveyard/games2.pkl", "rb"))
len(data)
train_df = data[:10000]
test_df = data[10000:12000]
print(len(train_df))
print(len(test_df))

10000
2000


In [3]:
# this method takes in all the moves in a game and encodes player, piece, file, rank, and modifiers
# `moves` is in the form {df}.loc[{game_index}, 'moves'].split()

def moves_vec(moves):

    move_vectors = np.zeros((100,5), dtype=int)
    
    player_idx = 0
    piece_idx = 1
    file_idx = 2
    rank_idx = 3
    mod_idx = 4

    piece_map = {'pawn': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5}
    file_map = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    rank_list = np.arange(1,9)
    mod_map = {'O-O-O': 0, 'O-O': 1, 'x': 2, '+': 3, '#': 4, '=': 5} # queenside castle, kingside castle, capture, check, mate, promo
    
    for i, move in enumerate(moves):
        if(i%2==0):                            # encode PLAYER as index 0
            move_vectors[i][player_idx] = 0    # white move
        else:
            move_vectors[i][player_idx] = 1    # black move
            
            
        for key in piece_map:                  # encode PIECE as index 1 
            if key in move:
                move_vectors[i][piece_idx]=piece_map[key]
                
        if move_vectors[i][piece_idx] == 100:
            move_vectors[i][piece_idx] = 0    # set pawns
                
                
        for key in file_map:                   # encode FILES A-B as integers 1-8
            if key in move:
                move_vectors[i][file_idx] = file_map[key]
                
                
        for rank in rank_list:                 # encode RANK 0-7 as integers 1-8
            if str(rank) in move:
                move_vectors[i][rank_idx] = int(rank) - 1 

                
        for key in mod_map:                    # encode queenside castle, kingside castle, capture, check, mate, promo
            if key in move:
                move_vectors[i][mod_idx] = mod_map[key]

    return move_vectors       # outputs a (10, 5) vector, numpy array with dtypes np.int64

# this method takes in the winner of the game and returns a one hot vector 
# `winner` is in the form {df}.loc[{game_index}, 'winner']

def one_hot(winner):
    if(winner == 'white'):
        return [1, 0]     # white win, black loss
    else:
        return [0, 1]     # white loss, black win
    
# test_game = train_df.loc[23, 'moves']
# type(test_game)
# test = moves_vec(test_game)

# print("color,\npiece: 'pawn': 0, 'N': 1, 'B': 2, 'R': 4, 'Q': 5, 'K': 6")
# print("file:'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8")
# print("moves string: ")
# print(test_game)
# print(test)

In [4]:
class Normalize(object):                                                                       
    def __init__(self):
        
        self.piece_max = 6
        self.file_max = 7
        self.rank_max = 7
        self.mod_max = 5
                                                                                               
    def __call__(self,sample):                                                                 
        sample = sample.type(torch.FloatTensor)
        # index 0 - PLAYER
        
        # index 1 - PIECE  
        sample[:,1] = sample[:,1] / self.piece_max
              
        # index 2 - FILE
        sample[:,2] = sample[:,2] / self.file_max
        
        # index 3 - RANK
        sample[:,3] = sample[:,3] / self.rank_max
        
        # index 4 - MODS
        sample[:,4] = sample[:,4] / self.mod_max

        return sample

In [5]:
class ChessDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Convert data to PyTorch tensors
        #embed()
        #moves = self.data.loc[idx, 'moves']
        moves = self.data.loc[idx, 'moves'].split()
        moves_tensor = torch.tensor(moves_vec(moves))
        moves_tensor = self.transform(moves_tensor)
        
        winner = self.data.loc[idx, 'winner']
        label = torch.tensor(one_hot(winner))
        label = label.type(torch.FloatTensor)
              
        return moves_tensor, label

# Create train and test data loaders
normalize_transform = Normalize()
train_data = ChessDataset(train_df, normalize_transform)
test_data = ChessDataset(test_df, normalize_transform)

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

#pull a batch from dataloaders manually by setting batch_size=1 and running the following:
batch = next(iter(train_loader))
print(batch)
print(train_loader)

[tensor([[[0.0000, 0.0000, 0.5714, 0.4286, 0.0000],
         [1.0000, 0.1667, 0.7143, 0.7143, 0.0000],
         [0.0000, 0.0000, 0.5714, 0.5714, 0.0000],
         [1.0000, 0.1667, 0.4286, 0.5714, 0.0000],
         [0.0000, 0.0000, 0.2857, 0.4286, 0.0000],
         [1.0000, 0.1667, 0.1429, 0.7143, 0.0000],
         [0.0000, 0.0000, 0.4286, 0.4286, 0.0000],
         [1.0000, 0.0000, 0.4286, 0.5714, 0.0000],
         [0.0000, 0.1667, 0.4286, 0.1429, 0.0000],
         [1.0000, 0.0000, 0.4286, 0.4286, 0.4000],
         [0.0000, 0.1667, 0.2857, 0.4286, 0.4000],
         [1.0000, 0.1667, 0.2857, 0.4286, 0.4000],
         [0.0000, 0.3333, 0.2857, 0.4286, 0.4000],
         [1.0000, 0.0000, 0.5714, 0.7143, 0.0000],
         [0.0000, 0.0000, 0.4286, 0.5714, 0.0000],
         [1.0000, 0.3333, 0.2857, 0.5714, 0.0000],
         [0.0000, 0.3333, 0.1429, 0.5714, 0.6000],
         [1.0000, 0.0000, 0.2857, 0.7143, 0.0000],
         [0.0000, 0.0000, 0.4286, 0.7143, 0.4000],
         [1.0000, 0.0000, 0.28

In [6]:
class ChessRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChessRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, num_layers=4)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, hidden = self.rnn(x)
        out = self.fc(hidden[-1])
        
        return out

In [7]:
# Set hyperparameters
input_size = 5 # number of features in the input (one-hot encoding of moves)
hidden_size = 64 # number of hidden units in the RNN
output_size = 2 # number of output classes (1 for each player)

learning_rate = 0.001
num_epochs = 20

model = ChessRNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    
    # Set model to train mode
    model.train()
    
    # Loop over the train data
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero out gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        #embed()
        print(f"outputs: {outputs.shape} {outputs.dtype}")
        print(f"labels: {labels.shape} {outputs.dtype}")
        loss = criterion(outputs, labels)

        

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update metrics
        #train_loss += loss.item() * inputs.size(0)
        train_loss += loss.item()
        #print(f"loss.item= {loss.item()}")
        #print(f"inputs.size={inputs.size(0)}")
        
        #train_acc += torch.sum(torch.argmax(outputs, axis=1) == torch.argmax(labels, axis = 1)).item()
        
    # Compute average loss and accuracy for the epoch
    train_loss = train_loss / len(train_loader.dataset)
    #train_acc = train_acc / len(train_loader.dataset)
    
    # Print training metrics for the epoch
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}'.format(
        epoch+1, num_epochs, train_loss, train_acc))


outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outputs: torch.Size([1, 2]) torch.float32
labels: torch.Size([1, 2]) torch.float32
outp

KeyboardInterrupt: 